In [ ]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.linear_model import LogisticRegression
import sys
from sklearn.model_selection import GridSearchCV
import seaborn as sns

In [ ]:
import sys
# Add the folder containing your patched file
sys.path.insert(0, '../boruta_py/boruta')

# Import your patched BorutaPy
from borutamar_py import BorutaPy

# Check that Python is using your local file
import borutamar_py as bp
print(bp.__file__)  
# should point to ../../boruta_py/boruta/borutamar_py.py
from eBoruta import eBoruta, TrialData, Features, Dataset, setup_logger


In [ ]:
from borutamar_py import BorutaPy 

In [ ]:
import os
print(os.getcwd())


In [ ]:
seed=42

In [ ]:
training_hc= pd.read_csv("../Training_residualshealthycontrol_healthycontrol.csv")

In [ ]:
training_neuro= pd.read_csv("../Training_residualshealthycontrol_neurodegenerative.csv")

In [ ]:
X_hc = training_hc.set_index("eid").drop(columns=training_hc.columns[-1])
y_hc = training_hc.set_index("eid")[training_hc.columns[-1]].astype("category")

In [ ]:
X_neuro = training_neuro.set_index("eid").drop(columns=training_neuro.columns[-1])
y_neuro = training_neuro.set_index("eid")[training_neuro.columns[-1]].astype("category")

In [ ]:
y_neuro

In [ ]:
results_neuro = pd.read_csv("Results/Feature_importances_LightGBM_balanced_Training_residualshealthycontrol_neurodegenerative.csv")

results_hc = pd.read_csv("Results/Feature_importances_LightGBM_balanced_Training_residualshealthycontrol_healthycontrol.csv")

In [ ]:
features_neuro = results_neuro[results_neuro["Importance"] !=0]["Feature"].unique().tolist()

In [ ]:
features_hc = results_hc[results_hc["Importance"] !=0]["Feature"].unique().tolist()

In [ ]:
clf = LogisticRegression(class_weight="balanced", solver='liblinear', random_state=seed)

In [ ]:
lasso_params = {
    'C': [0.01, 0.1, 1],
    'penalty' :['l1'],
    'solver' : ['liblinear'],
    'class_weight': ['balanced']
}




grid_search = GridSearchCV(
    estimator=clf,
    param_grid=lasso_params,
    cv=3,
    scoring="recall",
    n_jobs=4
)
grid_search.fit(X_hc[features_hc], y_hc)

best_model_hc = grid_search.best_estimator_
best_params_hc_logreg= grid_search.best_params_
print("Best CV recall:", grid_search.best_score_)

In [ ]:

grid_search.fit(X_neuro[features_neuro], y_neuro)

best_model_neuro = grid_search.best_estimator_
best_params_neuro_logreg= grid_search.best_params_
print("Best CV recall:", grid_search.best_score_)

In [ ]:
grid_search.best_params_

In [ ]:
lgb_params = {
    'n_estimators': [50, 100,200,500],
    'learning_rate': [0.01, 0.1],
    'max_depth': [1,5,7,10,15],
    'subsample': [0.01, 1],
    'colsample_bytree': [0.01, 0.1, 1.0],
    'reg_alpha': [0,0.1, 10],
    'reg_lambda': [0, 0.1, 10]
}

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
from sklearn.base import BaseEstimator

def get_imp(estimator):
    # equivalent to the builtin importance getter
    return estimator.feature_importances_


def get_permutation_imp(estimator: BaseEstimator, trial_data: TrialData) -> np.ndarray:
    imp = permutation_importance(
        estimator, trial_data.x_test, trial_data.y_test,
        scoring='accuracy', n_jobs=-1
    )
    return np.array(imp['importances_mean'])

In [ ]:
def plot_imp_history(df_history: pd.DataFrame):
    sns.lineplot(x='Step', y='Importance', hue='Feature', data=df_history)
    sns.lineplot(x='Step', y='Threshold', data=df_history, linestyle='--', linewidth=4)

In [ ]:
best_model_neuro

In [ ]:
boruta_neuro_logreg = eBoruta(
).fit(X_neuro[features_neuro], y_neuro.values, model_type=LogisticRegression,
    model_init_kwargs=
    best_params_neuro_logreg)
plot_imp_history(boruta.features_.history)

In [ ]:
features = boruta_neuro_logreg.features_
features.accepted, features.rejected, features.tentative

In [ ]:
boruta_hc_logreg = eBoruta(
).fit(
    X_hc[features_hc], y_hc.values, model_type=LogisticRegression,
    model_init_kwargs=best_params_hc_logreg)
plot_imp_history(boruta.features_.history)

In [ ]:
features_HC = boruta_HC.features_
features_HC.accepted, features_HC.rejected, features_HC.tentative

In [ ]:
lgb_params = {
    'n_estimators': [50, 100,200,500],
    'learning_rate': [0.01, 0.1],
    'max_depth': [1,5,7,10,15],
    'subsample': [0.01, 1],
    'colsample_bytree': [0.01, 0.1, 1.0],
    'reg_alpha': [0,0.1, 10],
    'reg_lambda': [0, 0.1, 10],
    'class_weight':['balanced'],
    'random_state':[42]
    
}

In [ ]:
lightgbm= lgb.LGBMClassifier(random_state=seed, class_weight='balanced', verbose=-1, n_jobs=4)

In [ ]:
grid_search = GridSearchCV(
    estimator=lightgbm,
    param_grid=lgb_params,
    cv=3,
    scoring="recall",
    n_jobs=4
)
grid_search.fit(X_neuro[features_hc], y_neuro)

best_lightgbm_neuro = grid_search.best_estimator_
best_params_neuro_lightgbm= grid_search.best_params_
print("Best CV recall:", grid_search.best_score_)

In [ ]:
grid_search = GridSearchCV(
    estimator=lightgbm,
    param_grid=lgb_params,
    cv=3,
    scoring="recall",
    n_jobs=4
)
grid_search.fit(X_hc[features_hc], y_hc)

best_lightgbm_hc = grid_search.best_estimator_
best_params_hc_lightgbm= grid_search.best_params_
print("Best CV recall:", grid_search.best_score_)

In [ ]:
best_params_hc_lightgbm

In [ ]:

boruta_neuro_lightgbm = eBoruta(
).fit(
    X_neuro[features_neuro], 
    y_neuro.to_numpy(), 
    model_type=lgb.LGBMClassifier, 
    model_init_kwargs=best_params_neuro_lightgbm
)

In [ ]:
plot_imp_history(boruta_neuro_lightgbm.features_.history)

In [ ]:
plot_imp_history(boruta_hc_lightgbm.features_.history)

In [ ]:

boruta_hc_lightgbm = eBoruta(
).fit(
    X_hc[features_hc], 
    y_hc.to_numpy(), 
    model_type=lgb.LGBMClassifier, 
    model_init_kwargs=best_params_hc_lightgbm
)

In [ ]:
features_HC_lightgbm = boruta_hc_lightgbm.features_
features_HC_lightgbm.accepted, features_HC_lightgbm.rejected, features_HC_lightgbm.tentative

In [ ]:
features_neuro_lightgbm = boruta_neuro_lightgbm.features_
features_neuro_lightgbm.accepted, features_neuro_lightgbm.rejected, features_neuro_lightgbm.tentative

In [ ]:
features_neuro_lightgbm.accepted.tolist()

In [ ]:
features_lightgbm_all =features_neuro_lightgbm.accepted.tolist() + features_HC_lightgbm.accepted.tolist()


In [ ]:
len(features_lightgbm_all)

In [ ]:
features_lightgbm_all

In [ ]:
features_logreg_all = features_HC.accepted.tolist() + features.accepted.tolist()

In [ ]:
len(features_logreg_all)

In [ ]:
features_logreg_all

In [ ]:
# Write to a file
with open("proteins_logreg_logregboruta_list.txt", "w") as f:
   for protein in features_logreg_all:
      f.write(protein + "\n")

In [ ]:
# Write to a file
with open("proteins_logreg_lightgbmboruta_list.txt", "w") as f:
   for protein in features_lightgbm_all:
      f.write(protein + "\n")